In [ ]:
import torch
import torchmetrics as tm

from config import *
from stages import *
from train import *

from data.util import crop_q_between, split_weekdays_and_weekends

%reload_ext autoreload
%autoreload 2

In [ ]:
mse = tm.MeanSquaredError().to(CONFIG.device)
mape = tm.MeanAbsolutePercentageError().to(CONFIG.device)

Load data

In [ ]:
mat_q = CONFIG.load('mat_q.pt')
mat_q.shape

Split data

##### Compare MSE and MAPE losses for different parameter configurations

Method 1: Pre-train RBMs + attach KELM

In [ ]:
results_1 = {}

for P in range(3125, 28125, 3125):
    CONFIG.spectral_threshold = P
    dbn = get_pre_trained_dbn(CONFIG, print_each=0)
    for N in range(4, 36, 4):
        CONFIG.dbn_hidden_layer_sizes = [N, N, N]

        mat_q_trend, mat_q_resid = preprocess_data(P, mat_q)
        mat_c, mat_x = compress_data(
            mat_q_resid.abs(), CONFIG.read_period, CONFIG.train_period, CONFIG.alpha)

        mat_c_wd, mat_c_we = split_weekdays_and_weekends(
            mat_c, CONFIG.train_period[0])

        mse_loss = mse_for_config(CONFIG, dbn, mat_c_we)

        print(f'P={P}, N={N}, Loss={mse_loss}')

Method 2: Pre-train DBN + Train DBN attaching KELM on each epoch

In [ ]:
results_1 = {}

for P in range(3125, 28125, 3125):
    CONFIG.spectral_threshold = P
    dbn = get_pre_trained_dbn(CONFIG, print_each=0)
    dbn_pretrained_state = dbn.state_dict()
    for N in range(4, 360, 40):
        CONFIG.dbn_hidden_layer_sizes = [N, N, N]

        mat_q_trend, mat_q_resid = preprocess_data(P, mat_q)
        mat_c, mat_x = compress_data(
            mat_q_resid.abs(), CONFIG.read_period, CONFIG.train_period, CONFIG.alpha)

        mat_c_wd, mat_c_we = split_weekdays_and_weekends(
            mat_c, CONFIG.train_period[0])

        dbn.load_state_dict(dbn_pretrained_state)
        mse_loss = mse_for_config(CONFIG, dbn, mat_c_we, dbn_training_epochs=100)

        print(f'P={P}, N={N}, Loss={mse_loss}')